In [65]:
from pycaret.time_series import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.utils.time_series import clean_time_index
import holidays
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_csv("daily_csv/export-jsps014-1d.csv", parse_dates=["timestamp"], index_col="timestamp")

# ลบค่าที่ซ้ำกัน
df = df[~df.index.duplicated(keep="last")]

# เติม timestamp ที่ขาดให้ครบทุกวัน
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
df = df.reindex(full_range)

# ใช้เฉพาะคอลัมน์ที่ต้องการ
df = df[['pm_2_5', 'humidity']]
df = df[(df['pm_2_5'] >= 0) & (df['pm_2_5'] <= 100)]

# เติมค่าที่ขาดด้วย Spline Interpolation
df['pm_2_5'] = df['pm_2_5'].interpolate(method='spline', order=2)
df['humidity'] = df['humidity'].interpolate(method='spline', order=2)

# เพิ่มฟีเจอร์ วัน เดือน ปี
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

# ฟังก์ชันระบุฤดูกาลของไทย
def get_thai_season(month):
    if month in [3, 4, 5]:
        return 'summer'
    elif month in [6, 7, 8, 9, 10]:
        return 'rainy'
    else:
        return 'winter'

# เพิ่มคอลัมน์ฤดูกาล
df['season'] = df['month'].apply(get_thai_season)
# One-hot encoding ฤดูกาล
df = pd.get_dummies(df, columns=['season'], prefix='season')

# แปลงค่า True/False เป็น 1/0
df[['season_rainy', 'season_summer', 'season_winter']] = df[['season_rainy', 'season_summer', 'season_winter']].astype(int)

# เพิ่มฟีเจอร์ "วันหยุด" ของประเทศไทย
thai_holidays = holidays.TH(years=range(df.index.year.min(), df.index.year.max() + 1))
df['is_holiday'] = df.index.to_series().apply(lambda x: 1 if x in thai_holidays else 0)

# แปลง DatetimeIndex เป็น PeriodIndex
df.index = df.index.to_period('D')

# ตรวจสอบค่าว่างในแต่ละคอลัมน์
print("Missing values in df:")
print(df.isna().sum())  # ควรแสดง 0 ในทุกคอลัมน์
print(df[df.isna().any(axis=1)])  # ควรไม่มีแถวที่แสดงผล

# แบ่งข้อมูลเป็น train set และ test set
train_size = len(df) - 15
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

# เติมช่วงเวลาที่ขาดหายไปใน train_df
expected_train_index = pd.period_range(start=train_df.index.min(), end=train_df.index.max(), freq='D')
train_df = train_df.reindex(expected_train_index)

# เติมค่าที่ขาดในคอลัมน์ day, month, year
train_df['day'] = train_df.index.day
train_df['month'] = train_df.index.month
train_df['year'] = train_df.index.year

# เติมค่าที่ขาดในคอลัมน์ฤดูกาล
train_df['season'] = train_df['month'].apply(get_thai_season)
train_df = pd.get_dummies(train_df, columns=['season'], prefix='season')
train_df[['season_rainy', 'season_summer', 'season_winter']] = train_df[['season_rainy', 'season_summer', 'season_winter']].fillna(0).astype(int)

# เติมค่าที่ขาดด้วย interpolation
train_df['pm_2_5'] = train_df['pm_2_5'].interpolate(method='linear')
train_df['humidity'] = train_df['humidity'].interpolate(method='linear')
train_df['is_holiday'] = train_df['is_holiday'].fillna(0)  # เติมค่าว่างใน is_holiday ด้วย 0

# ตรวจสอบ train_df อีกครั้ง
print("Missing values in train_df after filling missing values:")
print(train_df.isna().sum())
print(train_df[train_df.isna().any(axis=1)])

# แปลงข้อมูลด้วย Box-Cox (ต้องไม่มี NaN)
train_df['pm_2_5'], lambda_ = boxcox(train_df['pm_2_5'] + 1)

# ฟีเจอร์ที่ใช้ในโมเดล
features = ['is_holiday', 'humidity', 'season_rainy', 'season_summer', 'season_winter']
X_train = train_df[features]
X_test = test_df[features]

# ตรวจสอบดัชนีเวลา (index) ของ X_train และ X_test
print("X_train index type:", type(X_train.index))
print("X_test index type:", type(X_test.index))

# ตั้งค่า PyCaret
exp = TSForecastingExperiment()
exp.setup(
    data=train_df, 
    target='pm_2_5', 
    session_id=123,  
    fh=15, 
    use_gpu=True, 
    seasonal_period='D',
    numeric_imputation_target="ffill",  # เติมค่าที่ขาดด้วย forward fill
    enforce_exogenous=True  # ตั้งค่าให้รองรับ exogenous features
)

# สร้างโมเดล ARIMA
model = exp.create_model('arima', order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
model = exp.tune_model(model)
model = exp.finalize_model(model)

# ทำนายค่า pm_2_5
forecast = exp.predict_model(model, fh=15, X=X_test)

# แปลงค่าทำนายกลับด้วย inverse_boxcox
forecast['y_pred'] = inv_boxcox(forecast['y_pred'], lambda_) - 1

# ปรับค่าทำนายให้อยู่ในช่วงที่เหมาะสม
forecast['y_pred'] = np.clip(forecast['y_pred'], 0, 300)

# คำนวณ MAE และ RMSE
mae = mean_absolute_error(test_df['pm_2_5'], forecast['y_pred'])
rmse = np.sqrt(mean_squared_error(test_df['pm_2_5'], forecast['y_pred']))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

# พล็อตกราฟเปรียบเทียบค่าจริงกับค่าทำนาย
plt.figure(figsize=(10, 6))
plt.plot(df.index[-50:].to_timestamp(), df['pm_2_5'][-50:], label='Actual', marker='o')
plt.plot(forecast.index.to_timestamp(), forecast['y_pred'], label='Forecast', marker='s', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('pm_2_5')
plt.title('Actual vs Forecasted pm_2_5')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Missing values in df:
pm_2_5           0
humidity         0
day              0
month            0
year             0
season_rainy     0
season_summer    0
season_winter    0
is_holiday       0
dtype: int64
Empty DataFrame
Columns: [pm_2_5, humidity, day, month, year, season_rainy, season_summer, season_winter, is_holiday]
Index: []
Missing values in train_df after filling missing values:
pm_2_5           0
humidity         0
day              0
month            0
year             0
season_rainy     0
season_summer    0
season_winter    0
is_holiday       0
season_rainy     0
season_summer    0
season_winter    0
dtype: int64
Empty DataFrame
Columns: [pm_2_5, humidity, day, month, year, season_rainy, season_summer, season_winter, is_holiday, season_rainy, season_summer, season_winter]
Index: []
X_train index type: <class 'pandas.core.indexes.period.PeriodIndex'>
X_test index type: <class 'pandas.core.indexes.period.PeriodIndex'>


TypeError: X must be either None, or in an sktime compatible format, of scitype Series, Panel or Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynbIf you think X is already in an sktime supported input format, run sktime.datatypes.check_raise(X, mtype) to diagnose the error, where mtype is the string of the type specification you want for X. Possible mtype specification strings are as follows. "For Series scitype: ['pd.Series', 'np.ndarray', 'pd.DataFrame']. , "For Panel scitype: ['numpy3D', 'nested_univ', 'pd-long', 'df-list', 'pd-multiindex']. , "For Hierarchical scitype: ['pd_multiindex_hier']. 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ดึงค่าจริงจาก test_df
actual = test_df['pm_2_5']

# ดึงค่าทำนายจาก forecast
predicted = forecast['y_pred']

# คำนวณ MAE
mae = mean_absolute_error(actual, predicted)

# คำนวณ MSE
mse = mean_squared_error(actual, predicted)

# คำนวณ RMSE
rmse = np.sqrt(mse)

# คำนวณ MAPE
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

mape = calculate_mape(actual, predicted)

# คำนวณ R²
r2 = r2_score(actual, predicted)

# คำนวณความแม่นยำ (Accuracy)
mean_actual = np.mean(actual)
accuracy = (1 - (mae / mean_actual)) * 100

# แสดงผลลัพธ์
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(r2)
print(f"R² (ความแม่นยำ): {r2 * 100:.2f}%")
print(f"ความแม่นยำ (Accuracy): {accuracy:.2f}%")

MAE: 6.8774
MSE: 72.0011
RMSE: 8.4853
MAPE: 37.63%
-0.3828651828581682
R² (ความแม่นยำ): -38.29%
ความแม่นยำ (Accuracy): 62.28%
